# Introduction
- **Backgammon** - started on 5 Sep 2020.
- Wrote backgammon from scratch using pygame. 
- Developed simple AI algorithm.
* Credits:
    * Board image: https://www.vectorstock.com/royalty-free-vector/backgammon-game-vector-6036678
    * Icon image:  https://www.flaticon.com/free-icon/backgammon_2241312
    * Chip image:  https://www.flaticon.com/authors/freepik
    * Dice images:  https://www.flaticon.com/authors/smashicons
* Requirements:
    * ```!pip install pygame```
* Instructions:
    * Restart kernel and run all cells to start the game.
    * Left click a chip to move it. 
    * Right click a chip to move it by the smaller die value. 
    * Click anywhere to have the computer move. 
    * See outputs of the last cell for status during gameplay.
    * See logs directory for game history.
<br><br>
<a id="contents"></a>
- #### Table of contents:
    1. [Imports](#20200912132717)
    1. [Functions](#20200912133208)
        1. [Display](#20200912133233)
        1. [Gameplay](#20200912133325)
        1. [AI](#20200912133412)
    1. [Config](#20200912133440)
    1. [Init](#20200912133506)
    1. [Game loop](#20200912133533)
 

<!--1. [Config](#config)
    1. [Loading & Organization](#load)
    1. [Data QA](#qa)
    1. [Initial EDA](#eda)
    1. [Feature Engineering](#features)
    1. [Modeling](#model)
    1. [Hyper-Parameter Tweaking](#hyper)
    1. [Ensembling](#ensemble)
    1. [Prediction](#predict)
    1. [Submission, QA, Score](#submit)

     -->

<a id="20200912132717"></a>
# Imports
<!-- 1. [Imports](#20200912132717) -->
[Table of contents](#contents)

In [1]:
import pygame
import numpy as np
from datetime import datetime
import time, sys, socket, os

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


<a id="20200912133208"></a>
# Functions
<!-- 1. [Functions](#20200912133208) -->
[Table of contents](#contents)

<a id="20200912133233"></a>
<!-- 1. [Display](#20200912133233) -->
### Display
[Table of contents](#contents)

In [2]:
board_jail_X = 374
board_jail_Y = 320
board_edge_E = 716
board_edge_W = 32
board_edge_S = 622
board_edge_N = 18
piece_distance = 58.4
die_positions = [(150, 325), (210, 325), (90, 325), (270, 325)]

def board_location_to_pixel_x(loc):
    if loc == 0 or loc == 25:
        return board_jail_X
    if loc <= 6: # 1st Q
        return int(board_edge_E - (loc-1) * piece_distance)
    if loc <= 12:
        return int(board_edge_W + (12-loc) * piece_distance)
    if loc <= 18:
        return int(board_edge_W + (loc-13) * piece_distance)
    if loc <= 24:
        return int(board_edge_E + (loc-24) * piece_distance)

def board_location_chip_to_pixels(loc, chip):
    chip = int(np.round(chip))
    assert chip != 0, 'chips must not equal 0'
    chip_sign = np.sign(chip)
    pos_x = board_location_to_pixel_x(loc)
    offset = 0
    if 6 < abs(chip) < 12:
        offset = int(chip_size / 2.0)
    if abs(chip) > 6:
        chip = chip_sign * (((abs(chip)+3) % 5) + 1)
    
    if loc == 0 or loc == 25:
        pos_y = board_jail_Y + offset + chip * chip_size
        return (pos_x, pos_y)
    if loc <= 12:
        pos_y = board_edge_S - offset - (abs(chip)-1) * chip_size
        return (pos_x, pos_y)
    else:
        pos_y = board_edge_N + offset + (abs(chip)-1) * chip_size
        return (pos_x, pos_y)

In [3]:
def draw_board(screen, board_state):
    for index, chips in enumerate(board_state):
        if chips == 0:
            continue
        
        plot_chip = player_chip_dict[int(np.clip(np.round(chips), -1, 1))]
        
        for chip in range(1, abs(chips)+1):
            pos = board_location_chip_to_pixels(index, np.sign(chips)*chip)
            screen.blit(plot_chip, pos)
    return None

def draw_dice(screen, die_list, player_id):
    assert len(die_list) <= 4
    for i, die in enumerate(die_list):
        assert 1 <= die <= 6
        screen.blit(dice[player_name_dict[player_id]][die-1], die_positions[i])

def board_location_from_pixel(px, py):
    for index, chips in enumerate(board_state):
        if chips == 0:
            continue
        
        for chip in range(1, abs(chips)+1):
            pos = board_location_chip_to_pixels(index, np.sign(chips)*chip)
            if not (pos[0] <= px <= pos[0]+chip_size):
                continue
            if (pos[1] < py < pos[1]+chip_size):
                return index
    return None

<a id="20200912133325"></a>
<!-- 1. [Gameplay](#20200912133325) -->
### Gameplay
[Table of contents](#contents)

In [4]:
def get_default_initial_board():
    _board_state = np.array(
                 [0, 
                  2, 0, 0, 0, 0, -5, 
                  0, -3, 0, 0, 0, 5, 
                  -5, 0, 0, 0, 3, 0, 
                  5, 0, 0, 0, 0, -2, 
                  0, ])
    return list(_board_state)

In [5]:
def roll_dice():
    dval = sorted(list(np.random.choice(range(1, 7), 2)), reverse=True)
    if dval[0] == dval[1]:
        return [dval[0]]*4
    return dval

def check_win(board_state):
    if (np.array(board_state) >= 0).all():
        return True
    if (np.array(board_state) <= 0).all():
        return True
    
def get_winner_id(board_state):
    if (np.array(board_state) >= 0).all():
        return -1
    if (np.array(board_state) <= 0).all():
        return 1
    return None

In [6]:
def all_dice_options():
    result = list()
    for i in range(1, 7):
        for j in range(1, 7):
            if i == j:
                result.append([i, i, i, i])
            else:
                result.append([i, j])
    return result
const_all_dice_options = all_dice_options()

In [7]:
def is_legal_move_complete(board_state, move_loc, move_value, player_id, dice_list):
    if not is_legal_move(board_state, move_loc, move_value, player_id):
        return False
    
    legal_moves = find_legal_moves(board_state, dice_list, player_id, remove_duplicates=False)
    legal_moves = [v[0] for v in legal_moves]
    if (move_loc, move_value) in legal_moves:
        return True
    else:
        return False

def is_legal_move(board_state, move_loc, move_value, player_id):
    assert player_id in (player_id_south, player_id_north)
    
    # care only for south player. Treat north player by reversing the board.
    if player_id == player_id_north:
        return is_legal_move(-np.array(board_state[::-1]), len(board_state)-move_loc-1, 
                             move_value, player_id_south)
    
    players_occupancy = np.clip(board_state, -1, 1)
    
    assert player_id == player_id_south, 'player id must be that of south player'
    assert move_value > 0, 'move value must be positive'
    assert 1 <= move_value <= 6, 'move must be between 1 and 6'
    
    # moving from a positions not occupied is forbidden
    if players_occupancy[move_loc] != player_id:
        return False
    
    new_chip_position = max(move_loc - move_value, 0)
    
    # if eaten, must move eaten first
    if players_occupancy[-1] == player_id:
        if move_loc != len(players_occupancy)-1:
            return False
    
    # if removing chips from the board
    if new_chip_position == 0:
        # is player in a position to remove chips
        farthest_chip_value = (len(players_occupancy)-1) - ((players_occupancy == player_id_south)[::-1]).argmax()
        player_can_remove = farthest_chip_value <= 6
        if not player_can_remove:
            return False
        if move_value == move_loc:
            return True
        else:
            # attempting to remove from 5 with value 6 requires 5 to be the highest value position.
            return move_loc == farthest_chip_value
    
    # placing on empty is allowed
    if players_occupancy[new_chip_position] == 0:
        return True
    # placing on occupied position is allowed
    if players_occupancy[new_chip_position] == player_id_south:
        return True
    # eating is allowed
    if board_state[new_chip_position] == player_id_north:
        return True
    return False

def make_move(board_state, move_loc, move_value, player_id, check_legal=True):
    new_board = np.array(board_state).copy()
    
    if check_legal:
        is_legal = is_legal_move(board_state, move_loc, move_value, player_id)
        if not is_legal:
            print('Illegal move!')
            return None

    # player id south moves from top to bottom of the board state. 
    sign = -1 if player_id == player_id_south else 1
    
    new_chip_position = move_loc + sign * move_value
    new_chip_position = np.clip(new_chip_position, 0, len(new_board)-1)    
    # remove chip from origin position
    new_board[move_loc] -= player_id
    if ((new_chip_position == 0 and player_id == player_id_south)
       or (new_chip_position == len(board_state)-1 and player_id == player_id_north)): # if removing chip
        # do nothing. 
        pass
    elif new_board[new_chip_position] == -player_id:  # if eating chip
        new_board[new_chip_position] = player_id
        # add chip to jail
        if player_id == player_id_south:
            new_board[0] += player_id_north
        else:
            new_board[-1] += player_id_south
    elif 0 < new_chip_position < len(board_state)-1: # regular move
        new_board[new_chip_position] += player_id

    return new_board

def player_number_of_chips(board_state, player_id):
    return np.clip(np.array(board_state) * player_id_north, 0, None).sum()

def assert_board_state(board_state):
    assert len(board_state) == 26, "Board has 26 locations"
    assert board_state[ 0] * player_id_north >= 0, "Board position 0 is north player's jail, south player can't occupy it"
    assert board_state[-1] * player_id_south >= 0, "Board position -1 is south player's jail, north player can't occupy it"
    
    assert player_number_of_chips(board_state, player_id_north) <= 15, "North player has too many chips"
    assert player_number_of_chips(board_state, player_id_north) <= 15, "South player has too many chips"
    

In [8]:
def make_moves(board_state, move_list, player_id, check_legal=True):
    temp_board = np.array(board_state).copy()
    for move_loc, move_value in move_list:
        temp_board = make_move(temp_board, move_loc, move_value, player_id, check_legal=check_legal)
    return temp_board

def player_chip_value(board_state, player_id):
    if player_id == player_id_north:
        return player_chip_value(-np.array(board_state[::-1]), -player_id)
    assert player_id == player_id_south, "Player id must be south"
    return (np.clip(np.array(board_state) * player_id_south, 0, None) * np.array(range(26))).sum()

def board_score_naive(board_state, player_id):
    return player_chip_value(board_state, -player_id) - player_chip_value(board_state, player_id)

<a id="20200912133412"></a>
<!-- 1. [AI](#20200912133412) -->
### AI
[Table of contents](#contents)

In [9]:
def find_legal_moves(board_state, dice_list, player_id, remove_duplicates=True):
    all_locations = np.where(np.clip(board_state, -1, 1) == player_id)[0]
#     all_locations = np.where(np.array(board_state) * player_id >= 1)[0]
    legal_moves_list = list()
    for location in all_locations:
        for die in set(dice_list):
            if not is_legal_move(board_state, location, die, player_id):
                continue
            if len(dice_list) == 1:
                legal_moves_list += [[(location, die),]]
                continue
            temp_dice = dice_list.copy()
            temp_dice.remove(die)
            next_board = list(make_move(board_state, location, die, player_id, check_legal=False))
            
            next_moves = find_legal_moves(next_board, temp_dice, player_id)
            if next_moves == []:
                legal_moves_list += [[(location, die), ]]
            else:
                legal_moves_list += [[(location, die)] + move_ld for move_ld in next_moves]
    
    # make sure to use all the dice when possible. 
    if len(legal_moves_list):
        max_len = np.max([len(v) for v in legal_moves_list])
        legal_moves_list = [v for v in legal_moves_list if len(v) == max_len]
    
    if not remove_duplicates:
        return legal_moves_list
            
    # remove duplicate moves.
    trimmed_legal_moves_list = list()
    sorted_moves = list()
    for move in legal_moves_list:
        if sorted(move) in sorted_moves:
            continue
        else:
            trimmed_legal_moves_list.append(move)
            sorted_moves.append(sorted(move))

    return trimmed_legal_moves_list
    
# %timeit find_legal_moves(board_state, [1, 1, 1, 1], player_id = player_id_north)
# a = find_legal_moves(board_state, [1, 1, 1, 1], player_id = player_id_north)
# len(a)


In [10]:
def find_best_move(board_state, dice_list, player_id, scorer=board_score_naive):
    legal_moves = find_legal_moves(board_state, dice_list, player_id)
    if len(legal_moves) == 0:
        return [], scorer(board_state, player_id)
    legal_moves_short = list()
    board_states = list()
    for move in legal_moves:
        temp_board = list(make_moves(board_state, move, player_id, check_legal=False))
        if temp_board in board_states:
            continue
        legal_moves_short.append(move)
        board_states.append(temp_board)

    board_scores = [scorer(v, player_id) for v in board_states]
    best_score_idx = np.argmax(board_scores)

    return legal_moves_short[best_score_idx], board_scores[best_score_idx]

# start_time = int(time.time())
# find_best_move(board_state, [1, 1, 1, 1], player_id_north, scorer=board_score_v1)
# print('took', int(time.time())-start_time, 'sec')

In [11]:
def board_score_v1(board_state, player_id):
    board_scores = list()
    # find the score of the opponent's best move
    for dice_option in const_all_dice_options:
        _, next_score = find_best_move(board_state, dice_option, -player_id, scorer=board_score_naive)
        next_score = -next_score
        board_scores.append(next_score)
    score = np.mean(board_scores)
    
    lambd1, lambd2, lambd3, lambd4 = 0.000, 0.01, 0.001, 0.0000001
    # 1. penalty for the number of open houses
    if lambd1 != 0:
        score -= (np.array(board_state) == player_id).sum() * lambd1
    # 2. penalty for number chips
    if lambd2 != 0:
        score -= player_number_of_chips(board_state, player_id) * lambd2
    # 3. penalty for distance from home
    if lambd3 != 0:
        if player_id == player_id_south:
            score -= (np.clip(np.multiply(board_state, player_id), 0, None) * np.linspace(1, 1 + lambd3, len(board_state))).sum()
        else:
            score -= (np.clip(np.multiply(board_state[::-1], player_id), 0, None) * np.linspace(1, 1 + lambd3, len(board_state))).sum()
    # 4. reward for number of closed sets
    if lambd4 != 0:
        score += (np.multiply(board_state, player_id) >= 2).sum() * lambd4
        
    return score

# board_score_v1(board_state, player_id_south)

<a id="20200912133440"></a>
# Config
<!-- 1. [Config](#20200912133440) -->
[Table of contents](#contents)

In [12]:
if not os.path.exists('./logs/'):
    os.mkdir('./logs')

In [13]:
icon = pygame.image.load('./images/backgammon_icon1.png')
background = pygame.image.load('./images/board_8by7.jpg')
black_chip = pygame.image.load('./images/black_chip55.png')
red_chip = pygame.image.load('./images/red_chip55.png')
dice = {'red':   [pygame.image.load(f'./images/red_die{i}_50.png') for i in range(1, 7)], 
        'black': [pygame.image.load(f'./images/black_die{i}_50.png') for i in range(1, 7)]}

In [14]:
chip_size = 55
FPS = 24
display_size = (800, 700)

In [15]:
player_id_south = -1
player_id_north = -player_id_south
player_chip_dict = {player_id_south: black_chip, player_id_north: red_chip}
player_name_dict = {player_id_south: 'black', player_id_north: 'red'}

<a id="20200912133506"></a>
# Init
<!-- 1. [Init](#20200912133506) -->
[Table of contents](#contents)

In [16]:
# Initiallize the screen and clock
pygame.init();
screen = pygame.display.set_mode(display_size)
clock = pygame.time.Clock()

# title and Icon
pygame.display.set_caption('Backgammon')
pygame.display.set_icon(icon)

<a id="20200912133533"></a>
# Game loop
<!-- 1. [Game loop](#20200912133533) -->
[Table of contents](#contents)

In [21]:
dice_values = []
running=True
player_id = player_id_north
hold4input = True
board_state = get_default_initial_board()
log = list()
log.append(' - '.join(['Game Started', str(datetime.now()), f"{sys.version} on {socket.gethostname()}"]))
# game loop
while running:
    clock.tick(FPS)
    screen.blit(background, (0, 0))
    
    draw_board(screen, board_state)
    draw_dice(screen, dice_values, player_id)
    assert_board_state(board_state)
    
    if player_id == player_id_north and not hold4input:
        print("CPU playing")
        best_move, _ = find_best_move(board_state, dice_values, player_id, scorer=board_score_v1)
        for move_loc, move_value in best_move:
            board_state = make_move(board_state, move_loc, move_value, player_id)
            dice_values.remove(move_value)
            message = f"Player {player_name_dict[player_id]} made the move {(move_loc, move_value)}"
            log.append(' - '.join([str(datetime.now()), message]))
            log.append(' - '.join([str(datetime.now()), 'board state', str(board_state)]))

        
    for event in pygame.event.get():
                        
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONUP:
            hold4input = False
            if player_id == player_id_south:
                click_loc = board_location_from_pixel(*event.pos)
                if click_loc is None:
                    continue
                if event.button not in (1, 3):
                    continue
                
                move_value = dice_values[0] if event.button == 1 else dice_values[-1]
                
                if not is_legal_move_complete(board_state, click_loc, move_value, player_id, dice_values):
                    print('Illegal move!')
                    continue
                
                new_board = make_move(board_state, click_loc, move_value, player_id)
                if new_board is not None:
                        board_state = new_board
                        message = f"Player {player_name_dict[player_id]} made the move {(click_loc, move_value)}"
                        log.append(' - '.join([str(datetime.now()), message]))
                        log.append(' - '.join([str(datetime.now()), 'board state', str(board_state)]))
                        dice_values.remove(move_value)

    pygame.display.flip()

    if len(dice_values) == 0:
        dice_values = roll_dice()
        player_id = -player_id
        
        message = f"Next turn - {player_name_dict[player_id]} to play : {dice_values}"
        print(message)
        log.append(' - '.join([str(datetime.now()), message]))
        log.append(' - '.join([str(datetime.now()), 'board state', str(board_state)]))
        
        hold4input = True
        print("Waiting for input")
        continue
    
    player_stuck = ((len(find_legal_moves(board_state, dice_values, player_id)) == 0) 
                and len(dice_values) > 0)

    if check_win(board_state):
        
        message = f"Game won by {player_name_dict[get_winner_id(board_state)]} player!"
        print(message)
        log.append(' - '.join([str(datetime.now()), message]))
        log.append(' - '.join([str(datetime.now()), 'board state', str(board_state)]))
        
        break
    
    if player_stuck and not hold4input:
        
        message = f"Player {player_name_dict[player_id]} is stuck!"
        print(message)
        log.append(' - '.join([str(datetime.now()), message]))
        log.append(' - '.join([str(datetime.now()), 'board state', str(board_state)]))
        
        dice_values = []
        
        hold4input = True
        print("Waiting for input")
        
        continue

else:
    screen.fill((0, 0, 0))
    pygame.display.update()
    print('Game Over!')
#     pygame.display.quit()
#     pygame.quit()

with open(f"./logs/log v1 - {datetime.now().strftime('%Y_%m_%d___%H_%M_%S')}.txt", "w") as text_file:
    text_file.write('\n'.join(log))
    print('Log written')

# quit()

Next turn - black to play : [3, 2]
Waiting for input
Next turn - red to play : [6, 1]
Waiting for input
CPU playing
Next turn - black to play : [4, 3]
Waiting for input
Next turn - red to play : [6, 1]
Waiting for input
CPU playing
Next turn - black to play : [4, 4, 4, 4]
Waiting for input
Next turn - red to play : [6, 1]
Waiting for input
CPU playing
Next turn - black to play : [3, 1]
Waiting for input
Next turn - red to play : [2, 1]
Waiting for input
CPU playing
Next turn - black to play : [4, 3]
Waiting for input
Next turn - red to play : [3, 2]
Waiting for input
CPU playing
Next turn - black to play : [6, 6, 6, 6]
Waiting for input
Next turn - red to play : [5, 2]
Waiting for input
CPU playing
Next turn - black to play : [5, 4]
Waiting for input
Next turn - red to play : [5, 4]
Waiting for input
CPU playing
Next turn - black to play : [6, 6, 6, 6]
Waiting for input
Player black is stuck!
Waiting for input
Next turn - red to play : [4, 2]
Waiting for input
CPU playing
Next turn - b